In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import mplhep
from matplotlib.colors import LogNorm
plt.style.use(mplhep.style.CMS)

In [2]:
# run_ranges_periods, df_run_ranges, fiducial_cuts, fiducial_cuts_all, aperture_period_map, aperture_parametrisation, check_aperture, get_data, process_data_protons_multiRP
from processing import *

save_figures = False

Welcome to JupyROOT 6.22/06
{'2017B': 2.360904801, '2017C1': 5.313012839, '2017C2': 3.264135878, '2017D': 4.074723964, '2017E': 8.958810514, '2017F1': 1.708478656, '2017F2': 7.877903151, '2017F3': 3.632463163}
Luminosity: 37.190432966


In [3]:
help( process_data_protons_multiRP )

Help on function process_data_protons_multiRP in module processing:

process_data_protons_multiRP(df_protons_multiRP, df_ppstracks=None, apply_fiducial=True, within_aperture=False, random_protons=False, mix_protons=False, select_2protons=True, runOnMC=False)



In [ ]:
# def get_data( fileNames ):
    
#     df_protons_multiRP_list = []
#     df_protons_singleRP_list = []
#     df_ppstracks_list = []
#     df_counts_list = []

#     for file_ in fileNames:
#         print ( file_ )
#         with h5py.File( file_, 'r' ) as f:
#             print ( list(f.keys()) )

#             dset_protons_multiRP = f['protons_multiRP']
#             print ( dset_protons_multiRP.shape )
#             print ( dset_protons_multiRP[:,:] )

#             dset_protons_singleRP = f['protons_singleRP']
#             print ( dset_protons_singleRP.shape )
#             print ( dset_protons_singleRP[:,:] )

#             dset_ppstracks = f['ppstracks']
#             print ( dset_ppstracks.shape )
#             print ( dset_ppstracks[:,:] )

#             dset_columns_protons = f['columns_protons']
#             print ( dset_columns_protons.shape )
#             columns_protons = list( dset_columns_protons )
#             print ( columns_protons )
#             columns_protons_str = [ item.decode("utf-8") for item in columns_protons ]
#             print ( columns_protons_str )

#             dset_columns_ppstracks = f['columns_ppstracks']
#             print ( dset_columns_ppstracks.shape )
#             columns_ppstracks = list( dset_columns_ppstracks )
#             print ( columns_ppstracks )
#             columns_ppstracks_str = [ item.decode("utf-8") for item in columns_ppstracks ]
#             print ( columns_ppstracks_str )

#             dset_selections = f['selections']
#             selections_ = [ item.decode("utf-8") for item in dset_selections ]
#             print ( selections_ )

#             dset_counts = f['event_counts']
#             df_counts_list.append( pd.Series( dset_counts, index=selections_ ) )
#             print ( df_counts_list[-1] )

#             chunk_size = 1000000
#             entries_protons_multiRP = dset_protons_multiRP.shape[0]
#             start_ = list( range( 0, entries_protons_multiRP, chunk_size ) )
#             stop_ = start_[1:]
#             stop_.append( entries_protons_multiRP )
#             print ( start_ )
#             print ( stop_ )
#             for idx in range( len( start_ ) ):
#                 print ( start_[idx], stop_[idx] )
#                 #print ( dset[ start_[idx] : stop_[idx] ] )
#                 df_ = pd.DataFrame( dset_protons_multiRP[ start_[idx] : stop_[idx] ], columns=columns_protons_str ).astype(
#                                         { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
#                                           "muon0_charge": "int32",
#                                           "nVertices": "int32",
#                                           "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
#                                           "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32" } )
#                 df_protons_multiRP_list.append( df_ )
#                 print ( df_protons_multiRP_list[-1].head() )
#                 print ( "Data set size: {}".format( len( df_protons_multiRP_list[-1] ) ) )

#             entries_protons_singleRP = dset_protons_singleRP.shape[0]
#             start_ = list( range( 0, entries_protons_singleRP, chunk_size ) )
#             stop_ = start_[1:]
#             stop_.append( entries_protons_singleRP )
#             print ( start_ )
#             print ( stop_ )
#             for idx in range( len( start_ ) ):
#                 print ( start_[idx], stop_[idx] )
#                 #print ( dset[ start_[idx] : stop_[idx] ] )
#                 df_ = pd.DataFrame( dset_protons_singleRP[ start_[idx] : stop_[idx] ], columns=columns_protons_str ).astype(
#                                         { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
#                                           "muon0_charge": "int32",
#                                           "nVertices": "int32",
#                                           "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
#                                           "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32" } )
#                 df_protons_singleRP_list.append( df_ )
#                 print ( df_protons_singleRP_list[-1].head() )
#                 print ( "Data set size: {}".format( len( df_protons_singleRP_list[-1] ) ) )

#             entries_ppstracks = dset_ppstracks.shape[0]
#             start_ = list( range( 0, entries_ppstracks, chunk_size ) )
#             stop_ = start_[1:]
#             stop_.append( entries_ppstracks )
#             print ( start_ )
#             print ( stop_ )
#             for idx in range( len( start_ ) ):
#                 print ( start_[idx], stop_[idx] )
#                 #print ( dset[ start_[idx] : stop_[idx] ] )
#                 df_ = pd.DataFrame( dset_ppstracks[ start_[idx] : stop_[idx] ], columns=columns_ppstracks_str ).astype( { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "rpid": "int32" } )
#                 df_ppstracks_list.append( df_ )
#                 print ( df_ppstracks_list[-1].head() )
#                 print ( "Data set size: {}".format( len( df_ppstracks_list[-1] ) ) )

#     df_counts = df_counts_list[0]
#     for idx in range( 1, len( df_counts_list ) ):
#         df_counts = df_counts.add( df_counts_list[idx] )
#     print ( df_counts )

#     df_protons_multiRP = pd.concat( df_protons_multiRP_list )
#     print (df_protons_multiRP)

#     df_protons_singleRP = pd.concat( df_protons_singleRP_list )
#     print (df_protons_singleRP)

#     df_ppstracks = pd.concat( df_ppstracks_list )
#     print (df_ppstracks)
    
#     return (df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks)

In [ ]:
# run_ranges_periods = {}
# run_ranges_periods[ "2017B" ]  = (297020,299329)
# run_ranges_periods[ "2017C1" ] = (299337,300785)
# run_ranges_periods[ "2017C2" ] = (300806,302029)
# run_ranges_periods[ "2017D" ]  = (302030,303434)
# run_ranges_periods[ "2017E" ]  = (303435,304826)
# run_ranges_periods[ "2017F1" ] = (304911,305114)
# run_ranges_periods[ "2017F2" ] = (305178,305902)
# run_ranges_periods[ "2017F3" ] = (305965,306462)
# df_run_ranges = pd.DataFrame( run_ranges_periods, index=("min","max") ).transpose()

# aperture_period_map = {
#     "2016_preTS2"  : "2016_preTS2",
#     "2016_postTS2" : "2016_postTS2",
#     "2017B"        : "2017_preTS2",
#     "2017C1"       : "2017_preTS2",
#     "2017C2"       : "2017_preTS2",
#     "2017D"        : "2017_preTS2",
#     "2017E"        : "2017_postTS2",
#     "2017F1"       : "2017_postTS2",
#     "2017F2"       : "2017_postTS2",
#     "2017F3"       : "2017_postTS2",
#     "2018"         : "2018"
# }

In [ ]:
# # Per data period, arm=(0,1)
# # Periods: "2016_preTS2", "2016_postTS2", "2017_preTS2", "2017_postTS2", "2018"
# def aperture_parametrisation( period, arm, xangle, xi ):

#     #https://github.com/cms-sw/cmssw/tree/916cb3d20213734a0465240720c8c8c392b92eac/Validation/CTPPS/python/simu_config

#     if (period == "2016_preTS2"):
#         if   (arm == 0): return 3.76296E-05+((xi<0.117122)*0.00712775+(xi>=0.117122)*0.0148651)*(xi-0.117122);
#         elif (arm == 1): return 1.85954E-05+((xi<0.14324)*0.00475349+(xi>=0.14324)*0.00629514)*(xi-0.14324);
#     elif (period == "2016_postTS2"):
#         if   (arm == 0): return 6.10374E-05+((xi<0.113491)*0.00795942+(xi>=0.113491)*0.01935)*(xi-0.113491);
#         elif (arm == 1): return (xi-0.110)/130.0;
#     elif (period == "2017_preTS2"):
#         if   (arm == 0): return -(8.71198E-07*xangle-0.000134726)+((xi<(0.000264704*xangle+0.081951))*-(4.32065E-05*xangle-0.0130746)+(xi>=(0.000264704*xangle+0.081951))*-(0.000183472*xangle-0.0395241))*(xi-(0.000264704*xangle+0.081951));
#         elif (arm == 1): return 3.43116E-05+((xi<(0.000626936*xangle+0.061324))*0.00654394+(xi>=(0.000626936*xangle+0.061324))*-(0.000145164*xangle-0.0272919))*(xi-(0.000626936*xangle+0.061324));
#     elif (period == "2017_postTS2"):
#         if   (arm == 0): return -(8.92079E-07*xangle-0.000150214)+((xi<(0.000278622*xangle+0.0964383))*-(3.9541e-05*xangle-0.0115104)+(xi>=(0.000278622*xangle+0.0964383))*-(0.000108249*xangle-0.0249303))*(xi-(0.000278622*xangle+0.0964383));
#         elif (arm == 1): return 4.56961E-05+((xi<(0.00075625*xangle+0.0643361))*-(3.01107e-05*xangle-0.00985126)+(xi>=(0.00075625*xangle+0.0643361))*-(8.95437e-05*xangle-0.0169474))*(xi-(0.00075625*xangle+0.0643361));
#     elif (period == "2018"):
#         if   (arm == 0): return -(8.44219E-07*xangle-0.000100957)+((xi<(0.000247185*xangle+0.101599))*-(1.40289E-05*xangle-0.00727237)+(xi>=(0.000247185*xangle+0.101599))*-(0.000107811*xangle-0.0261867))*(xi-(0.000247185*xangle+0.101599));
#         elif (arm == 1): return -(-4.74758E-07*xangle+3.0881E-05)+((xi<(0.000727859*xangle+0.0722653))*-(2.43968E-05*xangle-0.0085461)+(xi>=(0.000727859*xangle+0.0722653))*-(7.19216E-05*xangle-0.0148267))*(xi-(0.000727859*xangle+0.0722653));
#     else:
#         return -999.

# def check_aperture( period, arm, xangle, xi, theta_x ):
#     return ( theta_x < -aperture_parametrisation( period, arm, xangle, xi ) )

In [ ]:
# def process_data_protons_multiRP( df_protons_multiRP, df_ppstracks=None, runOnMC=False ):

#     if not runOnMC:
#         df_protons_multiRP.loc[ :, "period" ] = np.nan
#         for idx_ in range( df_run_ranges.shape[0] ):
#             msk_period_ = ( ( df_protons_multiRP.loc[ :, "run" ] >= df_run_ranges.iloc[ idx_ ][ "min" ] ) & ( df_protons_multiRP.loc[ :, "run" ] <= df_run_ranges.iloc[ idx_ ][ "max" ] ) )
#             sum_period_ = np.sum( msk_period_ )
#             if sum_period_ > 0:
#                 period_key_ = df_run_ranges.index[ idx_ ]
#                 df_protons_multiRP.loc[ :, "period" ].loc[ msk_period_ ] = period_key_
#                 print ( "{}: {}".format( period_key_, sum_period_ ) )

#         df_protons_multiRP.loc[ :, "within_aperture" ] = df_protons_multiRP.apply(
#                 lambda row: check_aperture( aperture_period_map[ row["period"] ], row["arm"], 120., row["xi"], row["thx"] ), # FIXME
#                 axis=1
#                 )
    
#     df_protons_multiRP_index = df_protons_multiRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )

#     df_ppstracks_index = None
#     if not df_ppstracks is None:
#         df_ppstracks_index = df_ppstracks.set_index( ['run', 'lumiblock', 'event', 'slice'] )
        
#         df_protons_multiRP_index.loc[ :, "track_x1" ] = np.nan
#         df_protons_multiRP_index.loc[ :, "track_x2" ] = np.nan
#         df_protons_multiRP_index.loc[ :, "track_y1" ] = np.nan
#         df_protons_multiRP_index.loc[ :, "track_y2" ] = np.nan
        
#         df_protons_multiRP_index.loc[ :, "track_x1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 3 ]
#         df_protons_multiRP_index.loc[ :, "track_x2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 23 ]
#         df_protons_multiRP_index.loc[ :, "track_y1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 3 ]
#         df_protons_multiRP_index.loc[ :, "track_y2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 0 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 23 ]
#         df_protons_multiRP_index.loc[ :, "track_x1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 103 ]
#         df_protons_multiRP_index.loc[ :, "track_x2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "x"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 123 ]
#         df_protons_multiRP_index.loc[ :, "track_y1" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 103 ]
#         df_protons_multiRP_index.loc[ :, "track_y2" ].loc[ df_protons_multiRP_index.loc[ :, "arm" ] == 1 ] = df_ppstracks_index.loc[:, "y"].loc[ df_ppstracks_index.loc[ :, "rpid" ] == 123 ]

#     df_protons_multiRP_events = df_protons_multiRP_index.drop( columns=[ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm" ] )
#     df_protons_multiRP_events = df_protons_multiRP_events[ ~df_protons_multiRP_events.index.duplicated(keep='first') ]

#     df_protons_multiRP_events.loc[ :, "MX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: 13000. * np.sqrt( df_.iloc[0].xi * df_.iloc[1].xi ) )
#     df_protons_multiRP_events.loc[ :, "YX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: 0.5 * np.log( df_.iloc[0].xi / df_.iloc[1].xi ) )
#     df_protons_multiRP_events.loc[ :, "diffMWW_MX" ]  = df_protons_multiRP_events[ "recoMWW" ] - df_protons_multiRP_events[ "MX" ]
#     df_protons_multiRP_events.loc[ :, "ratioMWW_MX" ] = df_protons_multiRP_events[ "recoMWW" ] / df_protons_multiRP_events[ "MX" ]
#     df_protons_multiRP_events.loc[ :, "shiftedRatioMWW_MX" ] = df_protons_multiRP_events[ "ratioMWW_MX" ] - 1.
#     df_protons_multiRP_events.loc[ :, "diffYWW_YX" ]  = df_protons_multiRP_events[ "recoRapidityWW" ] - df_protons_multiRP_events[ "YX" ]
    
#     return (df_protons_multiRP_index, df_protons_multiRP_events, df_ppstracks_index)

### Signal

In [28]:
run_tables = False

labels_signals = [ "GGToWW-AQGC-A0W1e-6" ]
# labels_signals = [ "GGToWW-AQGC-A0W1e-6", "GGToWW-AQGC-A0W2e-6", "GGToWW-AQGC-A0W5e-6" ]

fileNames_signals = {
    "GGToWW-AQGC-A0W1e-6": [ "output-GGToWW-AQGC-A0W1e-6.h5" ],
    "GGToWW-AQGC-A0W2e-6": [ "output-GGToWW-AQGC-A0W2e-6.h5" ],
    "GGToWW-AQGC-A0W5e-6": [ "output-GGToWW-AQGC-A0W5e-6.h5" ]
    }

df_counts_signals = {}
df_signals_protons_multiRP_index = {}
df_signals_protons_multiRP_events = {}

for label_ in labels_signals:
    import time
    print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
    time_s_ = time.time()

    #df_counts_signal, df_protons_multiRP_signal, df_protons_singleRP_signal, df_ppstracks_signal = 4 * [None]
    #df_protons_multiRP_signal_index, df_protons_multiRP_signal_events, df_ppstracks_signal_index = 3 * [None]

    if run_tables:
        with pd.HDFStore( "reduced-data-store-{}.h5".format( label_ ), complevel=5 ) as store_:

            df_counts_signal_, df_protons_multiRP_signal_, df_protons_singleRP_signal_, df_ppstracks_signal_ = get_data( fileNames_signals[ label_ ] )
            df_protons_multiRP_signal_index_, df_protons_multiRP_signal_events_, df_ppstracks_signal_index_ = process_data_protons_multiRP( df_protons_multiRP_signal_, df_ppstracks_signal_, apply_fiducial=True, within_aperture=True, select_2protons=False, runOnMC=True )

            store_[ "counts" ] = df_counts_signal_
            store_[ "protons_multiRP"] = df_protons_multiRP_signal_index_
            store_[ "events_multiRP" ] = df_protons_multiRP_signal_events_
    
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label_ ), 'r' ) as store_:
        df_counts_signals[ label_ ] = store_[ "counts" ]
        df_signals_protons_multiRP_index[ label_ ] = store_[ "protons_multiRP" ]
        df_signals_protons_multiRP_events[ label_ ] = store_[ "events_multiRP" ]

    time_e_ = time.time()
    print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

2021/05/12 19:04:18
Total time elapsed: 1


In [29]:
for label_ in labels_signals:
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label_ ), 'r' ) as store_:
        print ( list( store_ ) )

['/counts', '/events_multiRP', '/protons_multiRP']


In [30]:
label_ = "GGToWW-AQGC-A0W1e-6"
df_counts_signals[ label_ ]

All       34159
Jet       34159
Muon      17498
Proton    12552
dtype: int64

In [31]:
df_signals_protons_multiRP_index[ label_ ]

crossingAngle        xi       thx       thy  \
run lumiblock event  slice                                                
1   2266      226549 0              120.0  0.049396  0.000070  0.000006   
              226551 0              120.0  0.047449  0.000007  0.000021   
                     0              120.0  0.051457  0.000022  0.000089   
    2267      226605 0              120.0  0.129090  0.000042 -0.000010   
              226690 0              120.0  0.028568  0.000061 -0.000014   
    2268      226716 0              150.0  0.164051 -0.000072 -0.000019   
              226773 0              150.0  0.044415 -0.000016  0.000041   
    2270      226932 0              130.0  0.032306  0.000006 -0.000075   
                     0              130.0  0.119526 -0.000012  0.000005   
    2273      227227 0              130.0  0.056667 -0.000025 -0.000072   
                     0              130.0  0.053138 -0.000011 -0.000024   
    2274      227321 0              150.0  0.082597  0.000042 -0.000018   
              227328 0              150.0  0.141799 -0.000062  0.000005   
              227324 0              150.0  0.057238 -0.000019  0.000045   
    2275      227457 0              150.0  0.068920 -0.000051  0.000024   
                     0              150.0  0.115064  0.000022  0.000018   
    2276      227546 0              140.0  0.090561 -0.000041  0.000076   
                     0              140.0  0.125299 -0.000021  0.000054   
              227579 0              140.0  0.118551 -0.000022 -0.000012   
                     0              140.0  0.169634 -0.000121 -0.000032   
    2277      227611 0              130.0  0.037509  0.000041  0.000004   
              227610 0              130.0  0.088627 -0.000125 -0.000004   
              227661 0              130.0  0.057884  0.000036  0.000051   
    2278      227745 0              150.0  0.084094 -0.000080 -0.000087   
    2279      227825 0              130.0  0.067304 -0.000003  0.000013   
              227868 0              130.0  0.062334  0.000017  0.000011   
    2280      227969 0              120.0  0.085506  0.000025  0.000011   
    2282      228117 0              130.0  0.037401  0.000059  0.000036   
                     0              130.0  0.058293  0.000015  0.000080   
              228121 0              130.0  0.100883 -0.000009 -0.000044   
...                                   ...       ...       ...       ...   
    9019      401833 0              130.0  0.128527 -0.000033 -0.000013   
              401874 0              130.0  0.033579 -0.000103 -0.000091   
                     0              130.0  0.084197  0.000007  0.000073   
    9021      402066 0              119.0  0.036611  0.000072  0.000042   
                     0              119.0  0.046874  0.000018  0.000021   
              402070 0              119.0  0.042629  0.000002  0.000039   
              402092 0              119.0  0.102647 -0.000046  0.000074   
    9024      402306 0              150.0  0.101640  0.000058  0.000090   
              402311 0              150.0  0.108578 -0.000074  0.000040   
              402331 0              150.0  0.107441  0.000044  0.000004   
              402381 0              150.0  0.100368 -0.000052 -0.000037   
    9026      402504 0              150.0  0.041278  0.000096 -0.000116   
    9039      403821 0              130.0  0.102213  0.000027  0.000032   
    9041      404017 0              130.0  0.057910 -0.000056 -0.000008   
              404092 0              130.0  0.075079  0.000072 -0.000071   
              404091 0              130.0  0.132769 -0.000061  0.000051   
    9042      404200 0              140.0  0.089057  0.000003  0.000123   
    9043      404249 0              140.0  0.099805  0.000084  0.000034   
    9049      404804 0              120.0  0.086237  0.000060  0.000043   
              404867 0              120.0  0.117212 -0.000054  0.000115   
              404859 0              120.0  0.069489 -

In [ ]:
df_signals_protons_multiRP_index[ label_ ][ [ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm", "random", "trackpixshift1", "rpid1", "trackpixshift2", "rpid2", "trackx1", "trackx2", "tracky1", "tracky2" ] ]

In [15]:
msk_columns_jet_ = [ key_[:len('jet')] == 'jet' for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_jet_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_jet_ ]
columns_ = [ "period" ]
columns_.extend( columns_jet_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

period     jet0_pt  jet0_eta  jet0_phi  \
run lumiblock event  slice                                           
1   2266      226549 0       2017D  418.121246 -0.146196  3.120644   
              226551 0      2017F3  203.932037 -1.086163  0.536344   
                     0      2017F3  203.932037 -1.086163  0.536344   
    2267      226605 0      2017F2  820.272827 -0.485583  0.133567   
              226690 0      2017C1  518.140259 -0.514777 -1.798476   
    2268      226716 0      2017F2  627.934326  1.253209  2.796034   
              226773 0       2017E  302.638733  0.741758 -1.999625   
    2270      226932 0       2017B  320.698212 -1.370161 -1.832831   
                     0       2017B  320.698212 -1.370161 -1.832831   
    2273      227227 0      2017F2  335.731934  0.259870 -3.108820   
                     0      2017F2  335.731934  0.259870 -3.108820   
    2274      227321 0      2017C1  488.559601 -0.576491 -2.253806   
              227328 0      2017C1  932.871216 -0.407201 -0.323714   
              227324 0       2017E  701.641846  0.654758 -0.564483   
    2275      227457 0       2017E  345.850189 -1.458291  3.099098   
                     0       2017E  345.850189 -1.458291  3.099098   
    2276      227546 0      2017C2  608.518616 -0.521754 -0.481351   
                     0      2017C2  608.518616 -0.521754 -0.481351   
              227579 0       2017E  819.906433  0.347934  1.723913   
                     0       2017E  819.906433  0.347934  1.723913   
    2277      227611 0      2017C1  528.839661 -0.532353  1.328756   
              227610 0      2017C2  529.333557 -1.337532 -2.574718   
              227661 0       2017D  282.744965  0.050019  0.024065   
    2278      227745 0       2017E  406.260406 -1.671813 -1.287626   
    2279      227825 0      2017C1  621.586121 -0.108547  0.472532   
              227868 0       2017E  380.339447 -0.829641 -0.163244   
    2280      227969 0       2017E  256.121185 -0.037238 -1.303618   
    2282      228117 0      2017C1  321.463196  0.091370 -2.196248   
                     0      2017C1  321.463196  0.091370 -2.196248   
              228121 0      2017C1  218.931900 -1.992268 -2.806273   
...                            ...         ...       ...       ...   
    9019      401833 0       2017E  935.048157 -0.001932 -2.990361   
              401874 0      2017C1  255.108627 -1.286281  0.929812   
                     0      2017C1  255.108627 -1.286281  0.929812   
    9021      402066 0       2017E  212.723862 -0.989809  3.071498   
                     0       2017E  212.723862 -0.989809  3.071498   
              402070 0       2017E  497.798157 -0.433670 -0.905242   
              402092 0       2017E  333.790253 -1.525020  1.502790   
    9024      402306 0      2017F3  963.704285 -0.359579  2.776858   
              402311 0       2017D  688.522278 -0.481343  1.212398   
              402331 0      2017F2  237.222641 -1.700881  2.371212   
              402381 0       2017D  836.326294 -0.134460 -1.515576   
    9026      402504 0       2017E  557.721985 -0.950084 -2.213205   
    9039      403821 0       2017D  619.208679 -0.424757 -1.993068   
    9041      404017 0       2017B  375.980621 -0.980524 -0.870988   
              404092 0       2017D  254.128647 -0.630076 -2.393038   
              404091 0      2017F2  371.837646 -1.045385  2.919649   
    9042      404200 0       2017E  234.823410  0.310485  1.554241   
    9043      404249 0      2017F2  200.956528 -2.329832  0.874255   
    9049      404804 0      2017F2  281.790100 -1.633022 -0.382359   
              404867 0      2017C1  760.376282  0.013739  2.162716   
              404859 0      2017F2  739.716553  0.816617  0.258207   
              404880 0       2017D  519.890320  0.528729  2.255377   
    9050      404965 0       2017E  547.413818 -0.213785 -0.337151   
              404983 0      2017F2  318.875214  1.334757 -1.272597   
    9051      405005 0      2017F3  29

In [22]:
msk_columns_jes_ = [ key_.find( 'jes' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_jes_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_jes_ ]
columns_ = [ "period" ]
columns_.extend( columns_jes_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

period  jet0_pt_jes_up  jet0_energy_jes_up  \
run lumiblock event  slice                                               
1   2266      226549 0       2017D      420.240908          433.124127   
              226551 0      2017F3      205.305596          344.082415   
                     0      2017F3      205.305596          344.082415   
    2267      226605 0      2017F2      824.436265          931.281134   
              226690 0      2017C1      520.420076          605.068608   
    2268      226716 0      2017F2      631.387965         1202.013633   
              226773 0       2017E      304.420995          402.331134   
    2270      226932 0       2017B      322.740635          680.581694   
                     0       2017B      322.740635          680.581694   
    2273      227227 0      2017F2      337.403261          364.399720   
                     0      2017F2      337.403261          364.399720   
    2274      227321 0      2017C1      490.709263          579.975773   
              227328 0      2017C1      938.051239         1024.829265   
              227324 0       2017E      705.563900          870.045350   
    2275      227457 0       2017E      348.774113          795.794598   
                     0       2017E      348.774113          795.794598   
    2276      227546 0      2017C2      611.236224          703.830340   
                     0      2017C2      611.236224          703.830340   
              227579 0       2017E      824.005310          881.263459   
                     0       2017E      824.005310          881.263459   
    2277      227611 0      2017C1      531.166555          619.419071   
              227610 0      2017C2      532.258071         1089.019796   
              227661 0       2017D      284.366650          290.482675   
    2278      227745 0       2017E      409.185482         1131.476825   
    2279      227825 0      2017C1      624.856339          638.580263   
              227868 0       2017E      382.192568          537.702717   
    2280      227969 0       2017E      257.664943          270.827514   
    2282      228117 0      2017C1      323.219535          341.869276   
                     0      2017C1      323.219535          341.869276   
              228121 0      2017C1      220.817071          827.644237   
...                            ...             ...                 ...   
    9019      401833 0       2017E      941.251382          949.969851   
              401874 0      2017C1      256.824509          501.526338   
                     0      2017C1      256.824509          501.526338   
    9021      402066 0       2017E      214.051718          332.552869   
                     0       2017E      214.051718          332.552869   
              402070 0       2017E      499.988469          557.788795   
              402092 0       2017E      336.624604          816.301906   
    9024      402306 0      2017F3      969.174482         1037.207336   
              402311 0       2017D      691.689511          782.495018   
              402331 0      2017F2      239.088083          681.867760   
              402381 0       2017D      841.468846          858.796360   
    9026      402504 0       2017E      560.236859          840.676769   
    9039      403821 0       2017D      621.981425          687.926649   
    9041      404017 0       2017B      377.820068          579.129045   
              404092 0       2017D      255.712790          312.601639   
              404091 0      2017F2      373.863675          598.531442   
    9042      404200 0       2017E      236.226055          254.630394   
    9043      404249 0      2017F2      203.333970         1055.769424   
    9049      404804 0      2017F2      283.936018          760.424774   
              404867 0      2017C1      764.788117          773.146638   
              404859 0      2017F2      743.419772         1013.303704   
              404880 0       201

In [ ]:
msk_columns_WLep_ = [ key_.find( 'WLep') >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_WLep_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_WLep_ ]
columns_ = [ "period", "recoMWlep" ]
columns_.extend( columns_WLep_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_WW_ = [ key_.find( 'WW') >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_WW_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_WW_ ]
columns_ = [ "period" ]
columns_.extend( columns_WW_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_xi_ = [ key_.find( 'xi' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_xi_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_xi_ ]
columns_ = [ "period" ]
columns_.extend( columns_xi_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [ ]:
msk_columns_eff_ = [ key_[:len('eff')] == 'eff' for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_eff_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_eff_ ]
columns_ = [ "period" ]
columns_.extend( columns_eff_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

In [32]:
msk_columns_sf_ = [ key_.find( 'sf' ) >= 0 for key_ in df_signals_protons_multiRP_index[ label_ ].columns ]
columns_sf_ = df_signals_protons_multiRP_index[ label_ ].columns[ msk_columns_sf_ ]
columns_ = [ "period" ]
columns_.extend( columns_sf_ )
df_signals_protons_multiRP_index[ label_ ][ columns_ ]

period  jet0_jer_sf  jet0_jer_sfup  \
run lumiblock event  slice                                       
1   2266      226549 0       2017D       1.1432         1.1654   
              226551 0      2017F3       1.0989         1.1444   
                     0      2017F3       1.0989         1.1444   
    2267      226605 0      2017F2       1.1432         1.1654   
              226690 0      2017C1       1.1432         1.1654   
    2268      226716 0      2017F2       1.1137         1.2533   
              226773 0       2017E       1.1815         1.2299   
    2270      226932 0       2017B       1.1432         1.1654   
                     0       2017B       1.1432         1.1654   
    2273      227227 0      2017F2       1.1432         1.1654   
                     0      2017F2       1.1432         1.1654   
    2274      227321 0      2017C1       1.1815         1.2299   
              227328 0      2017C1       1.1432         1.1654   
              227324 0       2017E       1.1815         1.2299   
    2275      227457 0       2017E       1.1307         1.2778   
                     0       2017E       1.1307         1.2778   
    2276      227546 0      2017C2       1.1432         1.1654   
                     0      2017C2       1.1432         1.1654   
              227579 0       2017E       1.1432         1.1654   
                     0       2017E       1.1432         1.1654   
    2277      227611 0      2017C1       1.1815         1.2299   
              227610 0      2017C2       1.1307         1.2778   
              227661 0       2017D       1.1432         1.1654   
    2278      227745 0       2017E       1.1307         1.2778   
    2279      227825 0      2017C1       1.1432         1.1654   
              227868 0       2017E       1.0989         1.1444   
    2280      227969 0       2017E       1.1432         1.1654   
    2282      228117 0      2017C1       1.1432         1.1654   
                     0      2017C1       1.1432         1.1654   
              228121 0      2017C1       1.2393         1.4301   
...                            ...          ...            ...   
    9019      401833 0       2017E       1.1432         1.1654   
              401874 0      2017C1       1.1137         1.2533   
                     0      2017C1       1.1137         1.2533   
    9021      402066 0       2017E       1.0989         1.1444   
                     0       2017E       1.0989         1.1444   
              402070 0       2017E       1.1432         1.1654   
              402092 0       2017E       1.1307         1.2778   
    9024      402306 0      2017F3       1.1432         1.1654   
              402311 0       2017D       1.1432         1.1654   
              402331 0      2017F2       1.1307         1.2778   
              402381 0       2017D       1.1432         1.1654   
    9026      402504 0       2017E       1.0989         1.1444   
    9039      403821 0       2017D       1.1432         1.1654   
    9041      404017 0       2017B       1.0989         1.1444   
              404092 0       2017D       1.1815         1.2299   
              404091 0      2017F2       1.0989         1.1444   
    9042      404200 0       2017E       1.1432         1.1654   
    9043      404249 0      2017F2       1.4085         1.6105   
    9049      404804 0      2017F2       1.1307         1.2778   
              404867 0      2017C1       1.1432         1.1654   
              404859 0      2017F2       1.0989         1.1444   
              404880 0       2017D       1.1815         1.2299   
    9050      404965 0       2017E       1.1432         1.1654   
              404983 0      2017F2       1.1307         1.2778   
    9051      405005 0      2017F3       1.2393         1.4301   
              405030 0      2017F3       1.1432         1.1654   
    9052      405137 0      2017C2       1.1432         1.1654   
    9057      405631 0       2017E       1.1432         1.1654   
  

In [ ]:
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_proton_all_weighted" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_multitrack_weighted" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_strictzero_weighted" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_proton_all" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_multitrack" ] ) )
print ( np.mean( df_signals_protons_multiRP_index[ label_ ][ "eff_strictzero" ] ) )

In [ ]:
df_protons_multiRP_groupby = df_signals_protons_multiRP_index[ label_ ][ [ "arm" ] ].groupby( ["run","lumiblock","event","slice"] )
msk_2protons = df_protons_multiRP_groupby[ "arm" ].transform( lambda s_: ( np.sum( s_ == 0 ) >= 1 ) & ( np.sum( s_ == 1 ) >= 1 ) )
msk_1proton_Arm0 = df_protons_multiRP_groupby[ "arm" ].transform( lambda s_: ( np.sum( s_ == 0 ) >= 1 ) & ( np.sum( s_ == 1 ) == 0 ) )
msk_1proton_Arm1 = df_protons_multiRP_groupby[ "arm" ].transform( lambda s_: ( np.sum( s_ == 0 ) == 0 ) & ( np.sum( s_ == 1 ) >= 1 ) )
print ( msk_2protons )
print ( msk_1proton_Arm0 )
print ( msk_1proton_Arm1 )

In [ ]:
df_signals_protons_multiRP_index[ label_ ][ msk_2protons ]
# df_signals_protons_multiRP_index[ label_ ][ msk_1proton_Arm0 ]
# df_signals_protons_multiRP_index[ label_ ][ msk_1proton_Arm1 ]

In [8]:
df_signals_protons_multiRP_events[ label_ ]

crossingAngle     jet0_pt  jet0_eta  jet0_phi  \
run lumiblock event  slice                                                  
1   2266      226551 0              120.0  203.932037 -1.086163  0.536344   
    2270      226932 0              130.0  320.698212 -1.370161 -1.832831   
    2273      227227 0              130.0  335.731934  0.259870 -3.108820   
    2275      227457 0              150.0  345.850189 -1.458291  3.099098   
    2276      227546 0              140.0  608.518616 -0.521754 -0.481351   
              227579 0              140.0  819.906433  0.347934  1.723913   
    2282      228117 0              130.0  321.463196  0.091370 -2.196248   
    2286      228503 0              130.0  649.005920 -0.110481  1.066668   
              228565 0              130.0  360.785767  0.943119 -1.373860   
    2289      228803 0              140.0  553.738098 -0.434034  1.881415   
    2291      229042 0              140.0  226.409943 -1.545674 -2.217598   
    2296      229520 0              120.0  359.400543  1.004407  1.754919   
    2297      229605 0              120.0  314.089844 -0.833665  2.804024   
    2298      229715 0              140.0  303.292603 -0.605599  1.378168   
    2299      229840 0              150.0  320.155548 -1.622710  1.047925   
    2301      230059 0              140.0  441.006378  0.965674 -0.766519   
    2302      230185 0              120.0  428.253479  0.875721 -2.115956   
    2305      230479 0              120.0  711.671204 -0.083028 -2.544181   
    2316      231571 0              130.0  304.978241 -1.147173  1.442453   
    2320      231972 0              120.0  776.558167 -0.373183  1.715490   
    2321      232050 0              130.0  288.986633 -0.175954 -0.479072   
    2325      232498 0              120.0  251.262024 -0.020143  0.202985   
    2326      232588 0              130.0  219.488464  1.728025  0.155746   
    2339      233842 0              130.0  356.559387 -0.401219 -1.865776   
    2351      235039 0              130.0  399.303131  0.779830  1.361783   
    4029      402810 0              130.0  218.588501  1.396799 -1.652311   
    4033      403253 0              130.0  302.841614 -0.415742 -0.754761   
    4034      403369 0              140.0  271.863373 -0.377092  2.046722   
    4042      404199 0              120.0  442.823151 -0.769472  0.117514   
    4046      404544 0              130.0  211.217300  0.388993  0.752271   
...                                   ...         ...       ...       ...   
    9046      404524 0              120.0  299.291779 -0.895165  1.838763   
    5691      69049  0              140.0  363.316864 -0.673153  0.781148   
              69094  0              140.0  342.928802 -1.205625  0.621981   
    5692      69173  0              150.0  212.847382  0.883428  2.801008   
    5693      69229  0              130.0  240.823746 -1.861926 -1.990464   
    6214      121345 0              120.0  238.723190  0.768451  1.406657   
    6217      121637 0              120.0  215.106949 -2.021043  1.333552   
    6229      122879 0              130.0  532.137878  0.928207  1.712423   
    6230      122966 0              120.0  284.843292 -0.112785 -1.573483   
    6235      123449 0              130.0  565.342529 -0.246432 -1.963999   
    6236      123509 0              130.0  215.110748  0.533007 -2.360625   
    6237      123616 0              130.0  264.498840  0.536307 -2.739888   
    6245      124442 0              120.0  304.889465 -1.116777 -2.110954   
              124479 0              120.0  632.597717 -0.139025 -0.905077   
    6248      124750 0              120.0  310.905151  0.663816  2.997675   
    6251      125087 0              140.0  410.933594  1.072456 -0.421513   
    6252      125109 0              150.0  411.727661 -0.547556 -2.763749   
              125137 0              150.0  318.394104  0.175385  2.942288   
    6255      125430 0              120.0  259.293335  0.715522 -1.694036   
    6469      146862 0  

In [23]:
msk_columns_MX_ = [ key_.find( 'MX' ) >= 0 for key_ in df_signals_protons_multiRP_events[ label_ ].columns ]
columns_MX_ = df_signals_protons_multiRP_events[ label_ ].columns[ msk_columns_MX_ ]
columns_ = [ "period" ]
columns_.extend( columns_MX_ )
df_signals_protons_multiRP_events[ label_ ][ columns_ ]

period           MX  diffMWW_MX  ratioMWW_MX  \
run lumiblock event  slice                                                 
1   2266      226551 0      2017F3   642.360352   86.471313     1.134615   
    2270      226932 0       2017B   807.822815   25.677490     1.031786   
    2273      227227 0      2017F2   713.363708    5.643250     1.007911   
    2275      227457 0       2017E  1157.675781   80.416260     1.069463   
    2276      227546 0      2017C2  1384.800293  -90.013062     0.934999   
              227579 0       2017E  1843.537598   38.018921     1.020623   
    2282      228117 0      2017C1   607.001648   16.448364     1.027098   
    2286      228503 0      2017C1  1325.282715  -54.555908     0.958835   
              228565 0       2017E   802.749634   58.867126     1.073332   
    2289      228803 0      2017F3  1227.886719  -93.609375     0.923764   
    2291      229042 0      2017C2   643.823792   67.639893     1.105060   
    2296      229520 0      2017F3  1005.956360   72.123718     1.071697   
    2297      229605 0      2017F3   675.655457   35.762207     1.052930   
    2298      229715 0      2017F3   706.302795   63.765625     1.090281   
    2299      229840 0      2017C2  1271.329590 -127.235962     0.899919   
    2301      230059 0       2017E  1236.970337  139.882935     1.113085   
    2302      230185 0      2017F3  1075.203979 -138.680237     0.871020   
    2305      230479 0      2017F2  1551.888184 -187.661865     0.879075   
    2316      231571 0       2017E   656.987549   36.510376     1.055572   
    2320      231972 0      2017F2  1483.320068   38.279297     1.025807   
    2321      232050 0      2017C1   641.582031  -11.267090     0.982439   
    2325      232498 0       2017D   471.837036   58.167297     1.123278   
    2326      232588 0       2017E   826.616821   83.408264     1.100903   
    2339      233842 0      2017F2   725.464417    5.818359     1.008020   
    2351      235039 0      2017C2  1186.753296  -38.437866     0.967611   
    4029      402810 0       2017E  1093.879761  -94.870972     0.913271   
    4033      403253 0       2017E   710.935303  -22.180115     0.968801   
    4034      403369 0      2017F1   538.999512   25.952332     1.048149   
    4042      404199 0       2017E   972.868591  101.248962     1.104073   
    4046      404544 0       2017D   530.055725  -85.518372     0.838662   
...                            ...          ...         ...          ...   
    9046      404524 0       2017B   741.818054    8.879150     1.011969   
    5691      69049  0       2017E   820.932617   -1.515076     0.998154   
              69094  0       2017E   709.815125  130.775513     1.184239   
    5692      69173  0      2017F2   660.591797  -95.704346     0.855123   
    5693      69229  0       2017D  1208.592529  -57.862793     0.952124   
    6214      121345 0       2017B   748.076538 -163.109436     0.781962   
    6217      121637 0      2017C2   887.822021 -214.539917     0.758353   
    6229      122879 0       2017E  1469.006348    0.226929     1.000154   
    6230      122966 0      2017F2   612.299683  -14.759888     0.975894   
    6235      123449 0      2017F2  1184.434570    2.286865     1.001931   
    6236      123509 0       2017E   523.942139  -45.725586     0.912728   
    6237      123616 0      2017C2   899.482605  -97.015564     0.892143   
    6245      124442 0      2017C2  1076.479858 -287.650085     0.732786   
              124479 0       2017D  1225.943848   83.500488     1.068111   
    6248      124750 0      2017C2   697.868042   23.354736     1.033466   
    6251      125087 0      2017F2  1207.338379 -106.293335     0.911961   
    6252      125109 0      2017F2   952.406189 -102.060242     0.892840   
              125137 0       2017E   926.122253  -50.521423     0.945448   
    6255      125430 0      2017F2   791.093201   29.141174     1.036837   
    6469      146862 0       2017D   561.236938   60.998718     1.108686   


In [24]:
msk_columns_YX_ = [ key_.find( 'YX' ) >= 0 for key_ in df_signals_protons_multiRP_events[ label_ ].columns ]
columns_YX_ = df_signals_protons_multiRP_events[ label_ ].columns[ msk_columns_YX_ ]
columns_ = [ "period" ]
columns_.extend( columns_YX_ )
df_signals_protons_multiRP_events[ label_ ][ columns_ ]

period        YX  diffYWW_YX    YX_nom  \
run lumiblock event  slice                                           
1   2266      226551 0      2017F3 -0.040549    0.089981 -0.040549   
    2270      226932 0       2017B -0.654139    0.014924 -0.654139   
    2273      227227 0      2017F2  0.032141   -0.021656  0.032141   
    2275      227457 0       2017E -0.256271   -0.020461 -0.256271   
    2276      227546 0      2017C2 -0.162342    0.002882 -0.162342   
              227579 0       2017E -0.179152    0.000554 -0.179152   
    2282      228117 0      2017C1 -0.221897    0.016659 -0.221897   
    2286      228503 0      2017C1 -0.043387   -0.015054 -0.043387   
              228565 0       2017E  0.354960   -0.002891  0.354960   
    2289      228803 0      2017F3 -0.519140    0.137671 -0.519140   
    2291      229042 0      2017C2 -0.473463   -0.053038 -0.473463   
    2296      229520 0      2017F3 -0.010036    0.057480 -0.010036   
    2297      229605 0      2017F3 -0.298634   -0.040324 -0.298634   
    2298      229715 0      2017F3  0.137830   -0.047110  0.137830   
    2299      229840 0      2017C2 -0.418647   -0.030150 -0.418647   
    2301      230059 0       2017E -0.106248    0.035250 -0.106248   
    2302      230185 0      2017F3  0.298712    0.144642  0.298712   
    2305      230479 0      2017F2 -0.161361   -0.014688 -0.161361   
    2316      231571 0       2017E -0.716215    0.071419 -0.716215   
    2320      231972 0      2017F2 -0.278201   -0.029214 -0.278201   
    2321      232050 0      2017C1 -0.485156    0.035705 -0.485156   
    2325      232498 0       2017D  0.032974    0.008672  0.032974   
    2326      232588 0       2017E  0.366834    0.022209  0.366834   
    2339      233842 0      2017F2 -0.446196    0.031060 -0.446196   
    2351      235039 0      2017C2 -0.214532    0.110076 -0.214532   
    4029      402810 0       2017E -0.086574    0.029669 -0.086574   
    4033      403253 0       2017E -0.313133    0.019478 -0.313133   
    4034      403369 0      2017F1 -0.015222   -0.031878 -0.015222   
    4042      404199 0       2017E -0.083812   -0.049524 -0.083812   
    4046      404544 0       2017D -0.238403    0.321529 -0.238403   
...                            ...       ...         ...       ...   
    9046      404524 0       2017B -0.176330   -0.082690 -0.176330   
    5691      69049  0       2017E -0.140655   -0.038950 -0.140655   
              69094  0       2017E -0.438079   -0.104805 -0.438079   
    5692      69173  0      2017F2  0.138003    0.049310  0.138003   
    5693      69229  0       2017D -0.235955   -0.109496 -0.235955   
    6214      121345 0       2017B -0.081526    0.117761 -0.081526   
    6217      121637 0      2017C2 -0.592100   -0.400207 -0.592100   
    6229      122879 0       2017E  0.046577    0.027604  0.046577   
    6230      122966 0      2017F2 -0.114747   -0.069812 -0.114747   
    6235      123449 0      2017F2  0.114652   -0.081124  0.114652   
    6236      123509 0       2017E -0.366120    0.545592 -0.366120   
    6237      123616 0      2017C2 -0.611597    0.185502 -0.611597   
    6245      124442 0      2017C2 -0.115846   -0.292813 -0.115846   
              124479 0       2017D -0.234027   -0.009472 -0.234027   
    6248      124750 0      2017C2 -0.014674    0.109336 -0.014674   
    6251      125087 0      2017F2  0.253878    0.024330  0.253878   
    6252      125109 0      2017F2 -0.255597    0.012465 -0.255597   
              125137 0       2017E -0.553995   -0.022268 -0.553995   
    6255      125430 0      2017F2 -0.416670    0.099278 -0.416670   
    6469      146862 0       2017D  0.107853   -0.067592  0.107853   
    6470      146958 0      2017C1  0.148966    0.000997  0.148966   
    6534      153363 0       2017E -0.020456   -0.018511 -0.020456   
    6552      155161 0      2017F3 -0.147367    0.065845 -0.147367   
    6547      154620 0      2017F2 -0.332124    0.017813 -0.332124   
    6554      155360 0       2017E -0.

In [ ]:
label_ = "GGToWW-AQGC-A0W5e-6"
df_protons_multiRP_signal_index_ = df_signals_protons_multiRP_index[ label_ ]

msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_signal_index_[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_signal_index_[ "arm" ] == 1 )

print ( len(df_protons_multiRP_signal_index_[ "xi" ][ msk1 ]), len(df_protons_multiRP_signal_index_[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 2, 2, figsize=(24,20) )
axes[0,0].hist2d( df_protons_multiRP_signal_index_[ "trackx1" ][ msk1 ], df_protons_multiRP_signal_index_[ "tracky1" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,0].set_xlabel( "X (mm)", fontsize=20 )
axes[0,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[0,1].hist2d( df_protons_multiRP_signal_index_[ "trackx1" ][ msk2 ], df_protons_multiRP_signal_index_[ "tracky1" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,1].set_xlabel( "X (mm)", fontsize=20 )
axes[0,1].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,0].hist2d( df_protons_multiRP_signal_index_[ "trackx2" ][ msk1 ], df_protons_multiRP_signal_index_[ "tracky2" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,0].set_xlabel( "X (mm)", fontsize=20 )
axes[1,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,1].hist2d( df_protons_multiRP_signal_index_[ "trackx2" ][ msk2 ], df_protons_multiRP_signal_index_[ "tracky2" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,1].set_xlabel( "X (mm)", fontsize=20 )
axes[1,1].set_ylabel( "Y (mm)", fontsize=20 )

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_signal_index_[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_signal_index_[ "arm" ] == 1 )

print ( len(df_protons_multiRP_signal_index_[ "xi" ][ msk1 ]), len(df_protons_multiRP_signal_index_[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
h_, xedges_, yedges_, im_ = axes[0].hist2d( df_protons_multiRP_signal_index_[ "xi" ][ msk1 ], df_protons_multiRP_signal_index_[ "thx" ][ msk1 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[0].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[0].set_xlabel( r"$\xi$", fontsize=20 )
axes[0].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
h_, xedges_, yedges_, im_ = axes[1].hist2d( df_protons_multiRP_signal_index_[ "xi" ][ msk2 ], df_protons_multiRP_signal_index_[ "thx" ][ msk2 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[1].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[1].set_xlabel( r"$\xi$", fontsize=20 )
axes[1].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )

### Background

In [ ]:
run_tables = True

resample_factor = 20
label = "data-random-resample_20"

fileNames_bkg = [
    "output-data-random-resample_20-2017B.h5",
    "output-data-random-resample_20-2017C.h5",
    "output-data-random-resample_20-2017D.h5",
    "output-data-random-resample_20-2017E.h5",
    "output-data-random-resample_20-2017F.h5"
]

import time
print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
time_s_ = time.time()

df_counts_bkg, df_protons_multiRP_bkg, df_protons_singleRP_bkg, df_ppstracks_bkg = 4 * [None]
df_protons_multiRP_bkg_index, df_protons_multiRP_bkg_events, df_ppstracks_bkg_index = 3 * [None]

if run_tables:
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), complevel=5 ) as store_:
        df_counts_bkg_list_ = []
        df_protons_multiRP_bkg_index_list_ = []
        df_protons_multiRP_bkg_events_list_ = []
        df_ppstracks_bkg_index_list_ = []
        for file_ in fileNames_bkg:
            df_counts_bkg_, df_protons_multiRP_bkg_, df_protons_singleRP_bkg_, df_ppstracks_bkg_ = get_data( [ file_ ] )
            df_protons_multiRP_bkg_index_, df_protons_multiRP_bkg_events_, df_ppstracks_bkg_index_ = process_data_protons_multiRP( df_protons_multiRP_bkg_, df_ppstracks_bkg_, apply_fiducial=True, within_aperture=True, random_protons=True, runOnMC=False )
            df_counts_bkg_list_.append( df_counts_bkg_ )
            df_protons_multiRP_bkg_index_list_.append( df_protons_multiRP_bkg_index_ )
            df_protons_multiRP_bkg_events_list_.append( df_protons_multiRP_bkg_events_ )
            df_ppstracks_bkg_index_list_.append( df_ppstracks_bkg_index_ )
            
        df_counts_bkg_ = df_counts_bkg_list_[0]
        for idx_ in range( 1, len( df_counts_bkg_list_ ) ):
            df_counts_bkg_ = df_counts_bkg_.add( df_counts_bkg_list_[ idx_ ] )

        df_protons_multiRP_bkg_index_ = pd.concat( df_protons_multiRP_bkg_index_list_ )
        df_protons_multiRP_bkg_events_ = pd.concat( df_protons_multiRP_bkg_events_list_ )
        df_ppstracks_bkg_index_ = pd.concat( df_ppstracks_bkg_index_list_ )
        
        store_[ "counts" ] = df_counts_bkg_
        store_[ "protons_multiRP"] = df_protons_multiRP_bkg_index_
        store_[ "events_multiRP" ] = df_protons_multiRP_bkg_events_

with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    df_counts_bkg = store_[ "counts" ]
    df_protons_multiRP_bkg_index = store_[ "protons_multiRP" ]
    df_protons_multiRP_bkg_events = store_[ "events_multiRP" ]
        
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

In [ ]:
with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    print ( list( store_ ) )

In [ ]:
df_counts_bkg

In [ ]:
df_protons_multiRP_bkg_index

In [ ]:
df_protons_multiRP_bkg_events

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_bkg_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_bkg_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_bkg_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_bkg_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 2, 2, figsize=(24,20) )
axes[0,0].hist2d( df_protons_multiRP_bkg_index[ "trackx1" ][ msk1 ], df_protons_multiRP_bkg_index[ "tracky1" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,0].set_xlabel( "X (mm)", fontsize=20 )
axes[0,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[0,1].hist2d( df_protons_multiRP_bkg_index[ "trackx1" ][ msk2 ], df_protons_multiRP_bkg_index[ "tracky1" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,1].set_xlabel( "X (mm)", fontsize=20 )
axes[0,1].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,0].hist2d( df_protons_multiRP_bkg_index[ "trackx2" ][ msk1 ], df_protons_multiRP_bkg_index[ "tracky2" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,0].set_xlabel( "X (mm)", fontsize=20 )
axes[1,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,1].hist2d( df_protons_multiRP_bkg_index[ "trackx2" ][ msk2 ], df_protons_multiRP_bkg_index[ "tracky2" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,1].set_xlabel( "X (mm)", fontsize=20 )
axes[1,1].set_ylabel( "Y (mm)", fontsize=20 )

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_bkg_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_bkg_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_bkg_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_bkg_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
h_, xedges_, yedges_, im_ = axes[0].hist2d( df_protons_multiRP_bkg_index[ "xi" ][ msk1 ], df_protons_multiRP_bkg_index[ "thx" ][ msk1 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[0].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[0].set_xlabel( r"$\xi$", fontsize=20 )
axes[0].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
h_, xedges_, yedges_, im_ = axes[1].hist2d( df_protons_multiRP_bkg_index[ "xi" ][ msk2 ], df_protons_multiRP_bkg_index[ "thx" ][ msk2 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[1].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[1].set_xlabel( r"$\xi$", fontsize=20 )
axes[1].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )

In [ ]:
labels_samples = {
    "GGToWW-AQGC-A0W1e-6": r"$\gamma\gamma \to WW \quad a_0^W=1e^{-6}$",
    "GGToWW-AQGC-A0W2e-6": r"$\gamma\gamma \to WW \quad a_0^W=2e^{-6}$",
    "GGToWW-AQGC-A0W5e-6": r"$\gamma\gamma \to WW \quad a_0^W=5e^{-6}$"
}

In [ ]:
fig, axes = plt.subplots( 2, 2, figsize=(24,2*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "jet0_mass", bins=100, range=(0.,250.), histtype="step", density=True, color="lightcoral", ax=axes[ row_, 0 ] )
df_protons_multiRP_signal_events.hist( "jet0_mass", bins=100, range=(0.,250.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 0 ] )

df_protons_multiRP_bkg_events.hist( "jet0_corrmass", bins=100, range=(0.,250.), histtype="step", density=True, color="lightcoral", ax=axes[ row_, 1 ] )
df_protons_multiRP_signal_events.hist( "jet0_corrmass", bins=100, range=(0.,250.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 1 ] )
row_ += 1

df_protons_multiRP_bkg_events.hist( "num_bjets_ak8", bins=10, range=(0.,10.), histtype="step", density=True, color="lightcoral", ax=axes[ row_, 0 ] )
df_protons_multiRP_signal_events.hist( "num_bjets_ak8", bins=10, range=(0.,10.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 0 ] )

df_protons_multiRP_bkg_events.hist( "num_bjets_ak4", bins=10, range=(0.,10.), histtype="step", density=True, color="lightcoral", ax=axes[ row_,  1 ] )
df_protons_multiRP_signal_events.hist( "num_bjets_ak4", bins=10, range=(0.,10.), histtype="step", density=True, color="darkblue", ax=axes[ row_, 1 ] )
row_ += 1


In [ ]:
msk_bkg = ( ( df_protons_multiRP_bkg_events.loc[ :, "jet0_corrmass"] >= 50.0 ) &
            ( df_protons_multiRP_bkg_events.loc[ :, "jet0_corrmass"] <= 110.0 ) & 
            ( df_protons_multiRP_bkg_events.loc[ :, "num_bjets_ak4"] == 0 ) )
msk_signal = ( ( df_protons_multiRP_signal_events.loc[ :, "jet0_corrmass"] >= 50.0 ) &
               ( df_protons_multiRP_signal_events.loc[ :, "jet0_corrmass"] <= 110.0 ) &
               ( df_protons_multiRP_signal_events.loc[ :, "num_bjets_ak4"] == 0 ) )
df_protons_multiRP_bkg_events = df_protons_multiRP_bkg_events.loc[ msk_bkg ]
df_protons_multiRP_signal_events = df_protons_multiRP_signal_events.loc[ msk_signal ]

In [ ]:
fig, axes = plt.subplots( 2, 2, figsize=(24,2*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "jet0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "jet0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$p_{\rm{T}} (\rm{jet}) \,\rm{(GeV)}$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "jet0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "jet0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$\eta (\rm{jet})$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

df_protons_multiRP_bkg_events.hist( "jet0_tau1", bins=50, range=(0.,1.), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "jet0_tau1", bins=50, range=(0.,1.), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$\tau_1 (\rm{jet})$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "jet0_tau2", bins=50, range=(0.,1.), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "jet0_tau2", bins=50, range=(0.,1.), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$\tau_2 (\rm{jet})$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Jet.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Jet.png", bbox_inches='tight' )

#  'run', 'lumiblock', 'event', 'slice', 'xi', 'thx', 'thy', 't', 'ismultirp', 'rpid', 'arm', 
#  'jet0_pt', 'jet0_eta', 'jet0_phi', 'jet0_energy', 'jet0_mass', 'jet0_corrmass', 'jet0_tau1', 'jet0_tau2', 'jet0_vertexz', 
#  'muon0_pt', 'muon0_eta', 'muon0_phi', 'muon0_energy', 'muon0_charge', 'muon0_iso', 'muon0_dxy', 'muon0_dz', 
#  'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 
#  'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi'

In [ ]:
fig, axes = plt.subplots( 2, 2, figsize=(24,2*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "muon0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "muon0_pt", bins=50, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$p_{\rm{T}} (\mu) \,\rm{(GeV)}$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "muon0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "muon0_eta", bins=50, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$\eta (\mu)$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

df_protons_multiRP_bkg_events.hist( "muon0_iso", bins=50, range=(0.,0.2), histtype="step", density=True, color="lightcoral", ax=axes[row_,0] )
df_protons_multiRP_signal_events.hist( "muon0_iso", bins=50, range=(0.,0.2), histtype="step", density=True, color="darkblue", ax=axes[row_,0] )
axes[ row_, 0 ].set_xlabel( r"$Isolation (\mu)$", fontsize=20 )
axes[ row_, 0 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "muon0_dz", bins=50, range=(0.,0.1), histtype="step", density=True, color="lightcoral", ax=axes[row_,1] )
df_protons_multiRP_signal_events.hist( "muon0_dz", bins=50, range=(0.,0.1), histtype="step", density=True, color="darkblue", ax=axes[row_,1] )
axes[ row_, 1 ].set_xlabel( r"$d_z (\mu) \,\rm{(cm)}$", fontsize=20 )
axes[ row_, 1 ].set_ylabel( "Density", fontsize=20 )
axes[ row_, 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Muon.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Muon.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "met", bins=50, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "met", bins=50, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$p_{\rm{T}}^{\rm{miss}} \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "met_phi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "met_phi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$\phi({\vec p}_{\rm{T}}^{\rm{miss}})$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_MET.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_MET.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "nVertices", bins=100, range=(0.,100.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "nVertices", bins=100, range=(0.,100.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( "Number of vertices", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "pfcand_nextracks", bins=100, range=(0.,100.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "pfcand_nextracks", bins=100, range=(0.,100.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( "Number of extra tracks", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Vertices_ExtraTracks.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Vertices_ExtraTracks.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "WLeptonicPt", bins=100, range=(0.,1000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "WLeptonicPt", bins=100, range=(0.,1000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$p_{\rm{T}} (W^{\rm{lep}}) \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "WLeptonicPhi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "WLeptonicPhi", bins=50, range=(-np.pi,np.pi), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$\phi (W^{\rm{lep}})$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_WLeptonic.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_WLeptonic.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "recoMWW", bins=100, range=(0.,2000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "recoMWW", bins=100, range=(0.,2000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$M_{WW} \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "recoRapidityWW", bins=100, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "recoRapidityWW", bins=100, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$Y_{WW}$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Mass_Rapidity_WW.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Mass_Rapidity_WW.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,1*10) )

row_ = 0
df_protons_multiRP_bkg_events.hist( "MX", bins=100, range=(0.,2000.), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "MX", bins=100, range=(0.,2000.), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$M_{\rm{pp}} \,\rm{(GeV)}$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "YX", bins=100, range=(-5.,5.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "YX", bins=100, range=(-5.,5.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$Y_{\rm{pp}}$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )
row_ += 1

if save_figures:
    plt.savefig( "plots/Variables_Mass_Rapidity_protons.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Mass_Rapidity_protons.png", bbox_inches='tight' )

In [ ]:
fig = plt.figure( figsize=(12,10) )

plt.hist( ( df_protons_multiRP_bkg_events[ "jet0_tau2" ] / df_protons_multiRP_bkg_events[ "jet0_tau1" ] ), bins=50, range=(0.,1.), histtype="step", density=True, color="lightcoral" )
plt.hist( ( df_protons_multiRP_signal_events[ "jet0_tau2" ] / df_protons_multiRP_signal_events[ "jet0_tau1" ] ), bins=50, range=(0.,1.), histtype="step", density=True, color="darkblue" )
plt.xlabel( r"$\tau_{21}$", fontsize=20 )
plt.ylabel( "Density", fontsize=20 )
plt.title( "" )

if save_figures:
    plt.savefig( "plots/Variables_tau21.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_tau21.png", bbox_inches='tight' )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )

df_protons_multiRP_bkg_events.hist( "shiftedRatioMWW_MX", bins=50, range=(-1.5,1.5), histtype="step", density=True, color="lightcoral", ax=axes[0] )
df_protons_multiRP_signal_events.hist( "shiftedRatioMWW_MX", bins=50, range=(-1.5,1.5), histtype="step", density=True, color="darkblue", ax=axes[0] )
axes[ 0 ].set_xlabel( r"$M_{WW} / M_{\rm{pp}} - 1$", fontsize=20 )
axes[ 0 ].set_ylabel( "Density", fontsize=20 )
axes[ 0 ].set_title( "" )

df_protons_multiRP_bkg_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), histtype="step", density=True, color="lightcoral", ax=axes[1] )
df_protons_multiRP_signal_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), histtype="step", density=True, color="darkblue", ax=axes[1] )
axes[ 1 ].set_xlabel( r"$Y_{WW} - Y_{\rm{pp}}$", fontsize=20 )
axes[ 1 ].set_ylabel( "Density", fontsize=20 )
axes[ 1 ].set_title( "" )

if save_figures:
    plt.savefig( "plots/Variables_Mass_Rapidity_difference.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Variables_Mass_Rapidity_difference.png", bbox_inches='tight' )

In [ ]:
msk_bkg_cut1 = ( np.abs( df_protons_multiRP_bkg_events[ "shiftedRatioMWW_MX" ] ) <= 0.30 )
msk_bkg_cut2 = ( np.abs( df_protons_multiRP_bkg_events[ "diffYWW_YX" ] ) <= 0.50 )
msk_bkg_A =  msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_B = ~msk_bkg_cut1 &  msk_bkg_cut2
msk_bkg_C =  msk_bkg_cut1 & ~msk_bkg_cut2
msk_bkg_D = ~msk_bkg_cut1 & ~msk_bkg_cut2

In [ ]:
x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(12,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='gray' )
plt.plot( (x_center,x_center), (y_min,y_max), color='gray' )
ax = plt.gca()
df_protons_multiRP_bkg_events[ msk_bkg_A ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightblue' )
df_protons_multiRP_bkg_events[ msk_bkg_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_bkg_events[ msk_bkg_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_bkg_events[ msk_bkg_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral' )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

n_events_bkg   = df_protons_multiRP_bkg_events.shape[0]
n_events_bkg_A = df_protons_multiRP_bkg_events[ msk_bkg_A ].shape[0]
n_events_bkg_B = df_protons_multiRP_bkg_events[ msk_bkg_B ].shape[0]
n_events_bkg_C = df_protons_multiRP_bkg_events[ msk_bkg_C ].shape[0]
n_events_bkg_D = df_protons_multiRP_bkg_events[ msk_bkg_D ].shape[0]
resample_factor = 20
print ( "Number of events: {}".format( n_events_bkg / resample_factor ) )
print ( "Number of events (A): {}".format( n_events_bkg_A / resample_factor ) )
print ( "Number of events (B): {}".format( n_events_bkg_B / resample_factor ) )
print ( "Number of events (C): {}".format( n_events_bkg_C / resample_factor ) )
print ( "Number of events (D): {}".format( n_events_bkg_D / resample_factor ) )

### Data

In [ ]:
run_tables = True

label = "data"

fileNames_data = [
    'output-data-2017B.h5',
    'output-data-2017C.h5',
    'output-data-2017D.h5',
    'output-data-2017E.h5',
    'output-data-2017F.h5'
]

import time
print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
time_s_ = time.time()

df_counts_data, df_protons_multiRP_data, df_protons_singleRP_data, df_ppstracks_data = 4 * [None]
df_protons_multiRP_data_index, df_protons_multiRP_data_events, df_ppstracks_data_index =  3 * [None]

if run_tables:
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), complevel=5 ) as store_:

        df_counts_data, df_protons_multiRP_data, df_protons_singleRP_data, df_ppstracks_data = get_data( fileNames_data )
        df_protons_multiRP_data_index, df_protons_multiRP_data_events, df_ppstracks_data_index = process_data_protons_multiRP( df_protons_multiRP_data, df_ppstracks_data, apply_fiducial=True, within_aperture=True, runOnMC=False )

        store_[ "counts" ] = df_counts_data
        store_[ "protons_multiRP"] = df_protons_multiRP_data_index
        store_[ "events_multiRP" ] = df_protons_multiRP_data_events

with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    df_counts_data = store_[ "counts" ]
    df_protons_multiRP_data_index = store_[ "protons_multiRP" ]
    df_protons_multiRP_data_events = store_[ "events_multiRP" ]
        
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

In [ ]:
with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    print ( list( store_ ) )

In [ ]:
df_counts_data

In [ ]:
df_protons_multiRP_data_index

In [ ]:
df_protons_multiRP_data_events

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_data_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_data_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_data_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_data_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 2, 2, figsize=(24,20) )
axes[0,0].hist2d( df_protons_multiRP_data_index[ "trackx1" ][ msk1 ], df_protons_multiRP_data_index[ "tracky1" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,0].set_xlabel( "X (mm)", fontsize=20 )
axes[0,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[0,1].hist2d( df_protons_multiRP_data_index[ "trackx1" ][ msk2 ], df_protons_multiRP_data_index[ "tracky1" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[0,1].set_xlabel( "X (mm)", fontsize=20 )
axes[0,1].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,0].hist2d( df_protons_multiRP_data_index[ "trackx2" ][ msk1 ], df_protons_multiRP_data_index[ "tracky2" ][ msk1 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,0].set_xlabel( "X (mm)", fontsize=20 )
axes[1,0].set_ylabel( "Y (mm)", fontsize=20 )
axes[1,1].hist2d( df_protons_multiRP_data_index[ "trackx2" ][ msk2 ], df_protons_multiRP_data_index[ "tracky2" ][ msk2 ], bins=(100,100), range=( (0.,25.), (-10.,10.) ), norm=LogNorm(), cmap='viridis' )
axes[1,1].set_xlabel( "X (mm)", fontsize=20 )
axes[1,1].set_ylabel( "Y (mm)", fontsize=20 )
if save_figures:
    plt.savefig( "plots/Hit_Map_Data_vs_Sector_Near_Far.pdf", bbox_inches='tight' )
    plt.savefig( "plots/Hit_Map_Data_vs_Sector_Near_Far.png", bbox_inches='tight' )

In [ ]:
msk1 = None
msk2 = None
msk1 = ( df_protons_multiRP_data_index[ "arm" ] == 0 )
msk2 = ( df_protons_multiRP_data_index[ "arm" ] == 1 )

print ( len(df_protons_multiRP_data_index[ "xi" ][ msk1 ]), len(df_protons_multiRP_data_index[ "xi" ][ msk2 ]) )

fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
h_, xedges_, yedges_, im_ = axes[0].hist2d( df_protons_multiRP_data_index[ "xi" ][ msk1 ], df_protons_multiRP_data_index[ "thx" ][ msk1 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[0].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[0].set_xlabel( r"$\xi$", fontsize=20 )
axes[0].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
h_, xedges_, yedges_, im_ = axes[1].hist2d( df_protons_multiRP_data_index[ "xi" ][ msk2 ], df_protons_multiRP_data_index[ "thx" ][ msk2 ], bins=(100,100), range=( (0.,0.25), (-300.e-06,300.e-06) ), norm=LogNorm(), cmap='viridis' )
bin_centres_x_ = ( xedges_[:-1] + xedges_[1:] ) / 2
bin_centres_y_ = ( yedges_[:-1] + yedges_[1:] ) / 2
axes[1].plot( bin_centres_x_, np.mean( (h_ * bin_centres_y_), axis=1) / np.mean( h_, axis=1) )
axes[1].set_xlabel( r"$\xi$", fontsize=20 )
axes[1].set_ylabel( r"$\theta_x^{\ast}$", fontsize=20 )
if save_figures:
    plt.savefig( "plots/ThetaX_vs_Xi_Data_vs_Sector.pdf", bbox_inches='tight' )
    plt.savefig( "plots/ThetaX_vs_Xi_Data_vs_Sector.png", bbox_inches='tight' )

In [ ]:
msk_data = ( ( df_protons_multiRP_data_events.loc[ :, "jet0_corrmass"] >= 50.0 ) & ( df_protons_multiRP_data_events.loc[ :, "jet0_corrmass"] <= 110.0 ) )
df_protons_multiRP_data_events = df_protons_multiRP_data_events.loc[ msk_data ]

In [ ]:
msk_data_cut1 = ( np.abs( df_protons_multiRP_data_events[ "shiftedRatioMWW_MX" ] ) <= 0.30 )
msk_data_cut2 = ( np.abs( df_protons_multiRP_data_events[ "diffYWW_YX" ] ) <= 0.50 )
msk_data_A =  msk_data_cut1 &  msk_data_cut2
msk_data_B = ~msk_data_cut1 &  msk_data_cut2
msk_data_C =  msk_data_cut1 & ~msk_data_cut2
msk_data_D = ~msk_data_cut1 & ~msk_data_cut2
msk_data = ~msk_data_A

In [ ]:
x_min = -1.5
x_max =  1.5
y_min = -3.0
y_max =  3.0
x_center = ( x_min + x_max ) / 2.
y_center = ( y_min + y_max ) / 2.

fig = plt.figure( figsize=(12,10) )
plt.plot( (x_min,x_max), (y_center,y_center), color='gray' )
plt.plot( (x_center,x_center), (y_min,y_max), color='gray' )
ax = plt.gca()
df_protons_multiRP_data_events[ msk_data_B ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_data_events[ msk_data_C ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='darkblue' )
df_protons_multiRP_data_events[ msk_data_D ].plot( "shiftedRatioMWW_MX", "diffYWW_YX", 'scatter', ax=ax, color='lightcoral' )
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

n_events_data_B = df_protons_multiRP_data_events[ msk_data_B ].shape[0]
n_events_data_C = df_protons_multiRP_data_events[ msk_data_C ].shape[0]
n_events_data_D = df_protons_multiRP_data_events[ msk_data_D ].shape[0]
resample_factor = 20
print ( "Number of events (B): {} - Ratio: {}".format( n_events_data_B, ( n_events_data_B / ( n_events_bkg_B / resample_factor ) ) ) )
print ( "Number of events (C): {} - Ratio: {}".format( n_events_data_C, ( n_events_data_C / ( n_events_bkg_C / resample_factor ) ) ) )
print ( "Number of events (D): {} - Ratio: {}".format( n_events_data_D, ( n_events_data_D / ( n_events_bkg_D / resample_factor ) ) ) )

### References